In [29]:
import os
import geojson
import itertools
import shutil
import shapely
import numpy as np
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union
import networkx as nx

In [7]:
def PolyMerge(geo_1, geo_2, save_path, save=True):
    # read geojson files
    with open(geo_1) as f:
        gj_1 = geojson.load(f)
        f.close()
    with open(geo_2) as f:
        gj_2 = geojson.load(f)
        f.close()
    x_shape = shape(gj_1['features'][0]['geometry'])
    y_shape = shape(gj_2['features'][0]['geometry'])
    if x_shape.intersects(y_shape):
        save_name = geo_1.split("\\")[-1].split('.')[0] + '_' + geo_2.split("\\")[-1].split('.')[0]+'.geojson'
        # print(f"Intersection found: {save_name}")
        merged_shape = x_shape.union(y_shape)
        merged_geojson = {
        'type': 'FeatureCollection',
        'features': [
                    {
                        'type': 'Feature',
                        'geometry': merged_shape.__geo_interface__,
                        'properties': {}
                    }
                ]
            }
        
        print(merged_geojson)
        # save into new geojson file
        if save:
            with open(os.path.join(save_path, save_name), 'w') as f:
                geojson.dump(merged_geojson, f)
                f.close()
        return 1
    else:
        # merged_shape = MultiPolygon([x_shape, y_shape])
        # print("No intersection")
        return 0

In [15]:
def PolyCollide(gj_1, gj_2):
    """
    This function takes two GeoJSON objects and returns True if they
    overlap.
    """
    x_shape = shape(gj_1['features'][0]['geometry'])
    y_shape = shape(gj_2['features'][0]['geometry'])
    return x_shape.intersects(y_shape)

In [4]:
data_dir = "final_footprints"
save_dir = "merged_footprints"
os.makedirs(save_dir, exist_ok=True)
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

In [11]:
def createCollisionMatrix(files):
    # Create a matrix to store collision information
    n_files = len(files)
    collision_matrix = np.zeros((n_files, n_files))

    # Loop through all pairs of files and check for collisions
    for i in range(n_files):
        for j in range(i+1, n_files):
            file1 = gpd.read_file(files[i])
            file2 = gpd.read_file(files[j])
            if not file1.empty and not file2.empty:
                if file1.intersects(file2).any():
                    collision_matrix[i,j] = 1
                    collision_matrix[j,i] = 1
    return collision_matrix

In [12]:
collision_matrix = createCollisionMatrix(files)
print(collision_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
"""
This code creates a graph, adds nodes for each file, and adds edges between 
nodes for files that have colliding objects. Then, it uses the 
connected_components function from networkx to find all the connected 
components in the graph, which represent groups of files with colliding 
objects. Finally, it prints out the groups.

Note that this approach can be slow for large numbers of files, 
since it checks every pair of files for collisions. 
If you have many files, you may want to consider a more efficient approach
that takes advantage of spatial indexing or other techniques to reduce 
the number of collision checks.


```
from rtree.index import Index

# create an R-tree index
idx = Index()

# add geometries to the index
for i, geom in enumerate(geometries):
    bbox = geom.bounds # get the bounding box of the geometry
    idx.insert(i, bbox) # insert the geometry ID and its bounding box into the index

# query the index for intersecting geometries
for i, geom in enumerate(geometries):
    bbox = geom.bounds
    results = idx.intersection(bbox, objects=True) # find the IDs of all intersecting geometries
    intersecting_geoms = [geometries[r.object] for r in results] # get the actual intersecting geometries
    # do something with the intersecting geometries...
```

"""

# create a graph
G = nx.Graph()

# add nodes to the graph
for file in files:
    G.add_node(file)

# add edges to the graph
for i in range(len(files)):
    for j in range(i+1, len(files)):
        file1, file2 = files[i], files[j]
        with open(file1) as f1, open(file2) as f2:
            data1, data2 = geojson.load(f1), geojson.load(f2)
            if PolyCollide(data1, data2):
                G.add_edge(file1, file2)

# find connected components
groups = list(nx.connected_components(G))
print(groups)
# print the groups
# for i, group in enumerate(groups):
#     print(f"Group {i+1}:")
#     for file in group:
#         print(file)
#     print()


[{'final_footprints\\0313103132221222220_0.png.geojson'}, {'final_footprints\\0313103132221222221_2.png.geojson', 'final_footprints\\0313103132221222230_2.png.geojson', 'final_footprints\\0313103132221222223_2.png.geojson', 'final_footprints\\0313103132223000000_3.png.geojson', 'final_footprints\\0313103132221222222_1.png.geojson', 'final_footprints\\0313103132221222220_1.png.geojson'}, {'final_footprints\\0313103132223000010_6.png.geojson', 'final_footprints\\0313103132223000010_2.png.geojson', 'final_footprints\\0313103132221222223_0.png.geojson', 'final_footprints\\0313103132221222233_1.png.geojson', 'final_footprints\\0313103132221222230_1.png.geojson', 'final_footprints\\0313103132223000100_3.png.geojson', 'final_footprints\\0313103132221222322_0.png.geojson', 'final_footprints\\0313103132221222232_3.png.geojson', 'final_footprints\\0313103132221222322_3.png.geojson', 'final_footprints\\0313103132223000011_3.png.geojson', 'final_footprints\\0313103132223000001_2.png.geojson', 'fin

In [32]:
def PolyMerge_v1(files, output_file, save=True):
    """
    Merge multiple geojson files into a single file
    """
    # Read all input files
    data_frames = [gpd.read_file(file) for file in files]
    
    # Merge the dataframes using unary_union function
    merged = unary_union(data_frames)
    merged_df = gpd.GeoDataFrame(geometry=[merged], crs=data_frames[0].crs)
    
    # Write to file if save=True
    if save:
        merged_df.to_file(output_file, driver='GeoJSON')
    else:
        return merged_df

def PolyMerge(geo_files, save_path, save_name, save=True):
    shapes = []
    for geo_file in geo_files:
        with open(geo_file) as f:
            gj = geojson.load(f)
            f.close()
        shape = shapely.geometry.shape(gj['features'][0]['geometry'])
        shapes.append(shape)
    merged_shape = shapely.ops.unary_union(shapes)
    
    merged_geojson = {
        'type': 'FeatureCollection',
        'features': [
            {
                'type': 'Feature',
                'geometry': merged_shape.__geo_interface__,
                'properties': {}
            }
        ]
    }
    if save:
        with open(os.path.join(save_path, save_name), 'w') as f:
            geojson.dump(merged_geojson, f)
            f.close()
    return 1 if merged_shape.is_valid else 0


In [33]:
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    # if length of group is one, copy the file into save destination
    if len(group) == 1:
        shutil.copy(list(group)[0], os.path.join(save_dir, f"group_{i+1}.geojson"))
        continue
    else:
        for file in group:
            print(file)
        print()
        PolyMerge(list(group), save_dir, f"group_{i+1}.geojson")

Group 1:
Group 2:
final_footprints\0313103132221222221_2.png.geojson
final_footprints\0313103132221222230_2.png.geojson
final_footprints\0313103132221222223_2.png.geojson
final_footprints\0313103132223000000_3.png.geojson
final_footprints\0313103132221222222_1.png.geojson
final_footprints\0313103132221222220_1.png.geojson

Group 3:
final_footprints\0313103132223000010_6.png.geojson
final_footprints\0313103132223000010_2.png.geojson
final_footprints\0313103132221222223_0.png.geojson
final_footprints\0313103132221222233_1.png.geojson
final_footprints\0313103132221222230_1.png.geojson
final_footprints\0313103132223000100_3.png.geojson
final_footprints\0313103132221222322_0.png.geojson
final_footprints\0313103132221222232_3.png.geojson
final_footprints\0313103132221222322_3.png.geojson
final_footprints\0313103132223000011_3.png.geojson
final_footprints\0313103132223000001_2.png.geojson
final_footprints\0313103132223000010_3.png.geojson
final_footprints\0313103132221222232_1.png.geojson
fin